# Bfastmonitor CPU using Python
## This is bfastmonitor Python

In [1]:
import os
import wget
import numpy as np
from datetime import datetime

from bfast import BFASTMonitor
from bfast.utils import crop_data_dates

import copy

import matplotlib
import matplotlib.pyplot as plt

#from sar so extra
import csv
from shutil import copyfile
import pandas as pd
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

import time
import ipywidgets as widgets

from ipyfilechooser import FileChooser
import folium

#specific imports
from functions import set_output_dir

In [2]:
%matplotlib inline

## Select a time series folder using ipyfilechooser. The standard download location is in downloads/time_series_name/[0,1,3]

### Optional: select a name for your output folder. 

In [3]:
from os.path import expanduser
path = expanduser("~")

def g(directory):
    return(directory)
                          
output_directory_chooser = widgets.interactive(g, 
                        directory=widgets.Text(description="Output storage name:", 
                                               style = {'description_width': 'initial'},
                                               placeholder = "output"))
file_chooser = FileChooser(path)

display(file_chooser)
display(output_directory_chooser)



FileChooser(path='/home/dven', filename='', show_hidden='False')

interactive(children=(Text(value='', description='Output storage name:', placeholder='output', style=Descripti…

In [7]:
timeseries_directory = file_chooser.selected
print(timeseries_directory)
set_output_dir(output_directory_chooser)


#timeseries_directory = "/home/dven/downloads/Time_series_2020-07-10_13-33-30/" #big (rwanda presumably)
#timeseries_directory = "/home/dven/downloads/Time_series_2020-07-07_11-51-35/1/" #small

None
Output directory name: derp


In [100]:
def get_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

print(get_size(start_path = timeseries_directory), 'bytes')

# 4926249639 bytes gave a memory error for m16, m48 could handle this.. but it takes long.. didn't let it run
# 6457220347 bytes 6.45 gb did not give a memory error for m16.. not sure why? maybe the new file structure? took 2334 seconds
# which is like 39 minutes

6457220347 bytes


# Load in data

In [71]:
start_time = time.time()

# set paths to data
time_series_path = os.path.join(timeseries_directory, 'stack.vrt') ## never start os path with a slash
dates_path = os.path.join(timeseries_directory, "dates.csv")

# load in time series and its metadata
time_series = gdal.Open(time_series_path)
raster_stack = time_series.ReadAsArray() # crashes for large data sets

#raster_stack = time_series.GetRasterBand(1).ReadAsArray() # get first date


# Save time
end_time = time.time()
print("Loading data took {} seconds.".format(end_time - start_time))

# Loading data took 2334.7296249866486 seconds on m16
# /home/dven/downloads/Time_series_2020-08-10_08-39-53_Rwanda/0/


Loading data took 4.472233533859253 seconds.


In [60]:
#np.save("./data/rwanda/data.npy", raster_stack)


# Store metadata

In [72]:
# Get metadata (geotransform contains )
geotransform = time_series.GetGeoTransform()

# Pixel size
xpixelsize = time_series.GetGeoTransform()[1]
ypixelsize = time_series.GetGeoTransform()[5]

# Coordinates
latitude = time_series.GetGeoTransform()[3]
longitude = time_series.GetGeoTransform()[0]

# Raster size
cols = time_series.RasterXSize
rows = time_series.RasterYSize

# Print to console
print(f"Number of  bands: {time_series.RasterCount}")
print(f"Number of columns: {cols}")
print(f"Number of  rows: {rows}")
print(f"Pixel size: {xpixelsize, ypixelsize}")
print(f"Latitude and longitude: {latitude,longitude}")

Number of  bands: 529
Number of columns: 14
Number of  rows: 8
Pixel size: (0.0002694945852358568, -0.0002694945852358568)
Latitude and longitude: (40.701497713586164, -8.420088821109099)


In [74]:
#time_series.GetProjectionRef()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [73]:
#Fabians code
data_orig = raster_stack

with open(dates_path) as f:
    dates_list = f.read().split('\n')
    dates = [datetime.strptime(d, '%Y-%m-%d') for d in dates_list if len(d) > 0]

# Set parameters

In [74]:
# parameters
k = 3
freq = 365
trend = False
hfrac = 0.25
level = 0.05
start_hist = datetime(2002, 1, 1)
start_monitor = datetime(2010, 1, 1)
end_monitor = datetime(2018, 1, 1)
position = (100,100)

print(dates)



[datetime.datetime(2000, 1, 8, 0, 0), datetime.datetime(2000, 1, 24, 0, 0), datetime.datetime(2000, 3, 28, 0, 0), datetime.datetime(2000, 5, 15, 0, 0), datetime.datetime(2000, 7, 18, 0, 0), datetime.datetime(2000, 9, 4, 0, 0), datetime.datetime(2000, 10, 22, 0, 0), datetime.datetime(2000, 11, 7, 0, 0), datetime.datetime(2000, 11, 23, 0, 0), datetime.datetime(2000, 1, 16, 0, 0), datetime.datetime(2000, 3, 4, 0, 0), datetime.datetime(2000, 3, 20, 0, 0), datetime.datetime(2000, 4, 5, 0, 0), datetime.datetime(2000, 5, 7, 0, 0), datetime.datetime(2000, 5, 23, 0, 0), datetime.datetime(2000, 6, 8, 0, 0), datetime.datetime(2000, 6, 24, 0, 0), datetime.datetime(2000, 7, 10, 0, 0), datetime.datetime(2000, 7, 26, 0, 0), datetime.datetime(2000, 8, 11, 0, 0), datetime.datetime(2000, 8, 27, 0, 0), datetime.datetime(2000, 9, 12, 0, 0), datetime.datetime(2000, 10, 14, 0, 0), datetime.datetime(2000, 11, 15, 0, 0), datetime.datetime(2000, 12, 1, 0, 0), datetime.datetime(2001, 2, 11, 0, 0), datetime.date

# Select monitoring period

In [75]:
start_date = dates[10] # 0 or 10 does not work.. 100 did
print("start monitoring period",start_date)
end_date = dates[-1]

def h(y):
    return(y)

pandas_dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in pandas_dates]
index = (0, len(options)-1)


start_end_date = widgets.interactive(h,
                                     y=widgets.SelectionRangeSlider(
                                            options=options,
                                            index=index,
                                            description='Select the monitoring date range: ',
                                            style = {'description_width': 'initial'},
                                            orientation='horizontal',
                                            layout={'width': '800px',"height":"200px"}))
display(start_end_date)

start monitoring period 2000-03-04 00:00:00


interactive(children=(SelectionRangeSlider(description='Select the monitoring date range: ', index=(0, 7400), …

In [92]:
start_monitor, end_monitor = start_end_date.result
print("start monitor: ",start_monitor)
print("end monitor: ",end_monitor)

start monitor:  2010-10-25 00:00:00
end monitor:  2020-06-07 00:00:00


In [93]:
data, dates = crop_data_dates(data_orig, dates, start_hist, end_monitor)
print("First date: {}".format(dates[0]))
print("Last date: {}".format(dates[-1]))
print("Shape of data array: {}".format(data.shape))

First date: 2002-01-29 00:00:00
Last date: 2020-06-07 00:00:00
Shape of data array: (473, 8, 14)


# Run bfastmonitor()

In [94]:
# fit BFAST using the CPU implementation (single pixel)
from numpy import nan

start_time = time.time()


model = BFASTMonitor(
            start_monitor,
            freq=freq,
            k=k,
            hfrac=hfrac,
            trend=trend,
            level=level,
            backend='python',
            verbose=1
            )


# only apply on a small subset
data = data[:,:200,:200]
model.fit(data, dates, nan_value=nan) #changed to nan

end_time = time.time()
print("Fitting model took {} seconds.".format(end_time - start_time))


Processing row 0
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Fitting model took 0.6198084354400635 seconds.


# Select negative magnitudes

In [82]:
# select only negative magnitudes
breaks = model.breaks # index of date that has a break in dates
means = model.means # magnitudes of breaks

no_breaks_indices = (breaks == -1)
means[no_breaks_indices] = 0
means[means > 0] = 0 # only want negative mean changes

breaks_plot = breaks.astype(np.float)
breaks_plot[breaks == -2] = np.nan
breaks_plot[breaks == -1] = np.nan
breaks_plot[means >= 0] = np.nan


In [83]:
dates_monitor = []

# collect dates for monitor period
for i in range(len(dates)):
    if start_monitor <= dates[i]:
        dates_monitor.append(dates[i])
dates_array = np.array(dates_monitor) # dates_array is the dates that are in the monitoring period


# Classify output for plotting

In [84]:
idx_starts = {}

# this gives the index of all the data points in the year and after
for year in range(start_monitor.year,end_monitor.year):
    idx_starts[year] = np.argmax((dates_array >= datetime(year, 1, 1)) > False) 
    
print(idx_starts)
breaks_plot_years = copy.deepcopy(breaks_plot)

#classifying for plotting
ticklist=[]
for idx, year in enumerate(idx_starts):
    ticklist.append(str(year))

    if idx == 0:
        breaks_plot_years[breaks_plot <= idx_starts[year+1]] = 0
        continue
    
    if idx == len(idx_starts)-1:
        breaks_plot_years[np.where(idx_starts[year] < breaks_plot)] = len(idx_starts)-1
        continue
        
    breaks_plot_years[np.where(np.logical_and(idx_starts[year] < breaks_plot, breaks_plot <= idx_starts[year+1]))] = idx
print(ticklist)
print(breaks_plot_years)

{2018: 0, 2019: 9}
['2018', '2019']
[[ 0.  0. nan nan  0.  0.  1.  1. nan nan  0.  0.  0.  0.]
 [ 0. nan nan nan  0.  0. nan nan nan nan  0.  0.  0.  0.]
 [nan nan nan nan nan  0. nan nan nan  0.  0.  0.  0.  0.]
 [ 0. nan nan nan nan  0.  0.  0. nan  0.  0.  0.  0.  0.]
 [nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [nan nan nan nan  0.  0.  0.  0.  0.  0. nan nan nan nan]
 [nan nan  0.  0. nan nan  0.  0.  0.  0. nan nan nan  0.]
 [nan  0.  0.  0. nan nan nan nan  0.  0. nan nan nan  0.]]


# Plot output

In [85]:
bins = len(idx_starts)

cmap = plt.get_cmap("rainbow")
cmaplist = [cmap(i) for i in range(cmap.N)]
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

bounds = np.linspace(0, bins-1, bins) #third number is the amount of bins in the colorbar 0=0, 6 = ncolors, 7= nyears
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
im = axes.imshow(breaks_plot_years, cmap=cmap, vmin=0, vmax=bins, norm=norm)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax, ticks=range(bins))
labels = cbar_ax.set_yticklabels(ticklist)

plt.savefig("./output/picture.png")
plt.show()


ZeroDivisionError: division by zero

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f1316591488> (for post_execute):


ZeroDivisionError: division by zero

ZeroDivisionError: division by zero

<Figure size 720x720 with 2 Axes>

In [53]:
# #this doesn't work yet

# output_raster = gdal.GetDriverByName('GTiff').Create('myraster.tif',cols, rows, 1 ,gdal.GDT_Float32)  # Open the file

# output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
# srs = osr.SpatialReference()                 # Establish its coordinate encoding
# srs.ImportFromEPSG(4326)                     # This one specifies WGS84 lat long.
#                                              # Anyone know how to specify the 
#                                              # IAU2000:49900 Mars encoding?
# output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system 
#                                                    # to the file
# output_raster.GetRasterBand(1).WriteArray(breaks_plot_years)   # Writes my array to the raster

# output_raster.FlushCache()

# output_raster

In [54]:
print(breaks_plot_years)
breaks_plot_years_norm = 1/(breaks_plot_years + 0.01)
print(breaks_plot_years_norm)

[[ 8.  4.  0.  0.  0.  9.  4.  4.  0.  0.  0.  0.  0.  0.]
 [ 8.  5.  0.  0.  0.  9. 11. 11.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  8.  3.  3.  1.  0.  2.  2.  3.  3.]
 [ 1.  1.  1.  1.  1.  6.  6.  6.  6.  1.  6.  6.  3.  3.]
 [ 1.  1.  1.  1.  1.  7.  6.  6.  6.  7.  7.  7.  5.  3.]
 [ 1.  1.  1.  1.  1.  1.  7.  7.  8. nan  7.  7.  2.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  5.  5.  2.  2.  1.  1.]
 [nan  1.  8.  8.  3.  4.  1.  1.  5.  5.  2.  2.  1.  1.]]
[[1.24843945e-01 2.49376559e-01 1.00000000e+02 1.00000000e+02
  1.00000000e+02 1.10987791e-01 2.49376559e-01 2.49376559e-01
  1.00000000e+02 1.00000000e+02 1.00000000e+02 1.00000000e+02
  1.00000000e+02 1.00000000e+02]
 [1.24843945e-01 1.99600798e-01 1.00000000e+02 1.00000000e+02
  1.00000000e+02 1.10987791e-01 9.08265213e-02 9.08265213e-02
  1.00000000e+02 1.00000000e+02 1.00000000e+02 1.00000000e+02
  1.00000000e+02 1.00000000e+02]
 [9.90099010e-01 9.90099010e-01 9.90099010e-01 9.90099010e-01
  9.90099010e-01 1.24843945

In [55]:
#cmap = plt.get_cmap("Oranges")

#cmaplist = [cmap(i) for i in range(cmap.N)]

#cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# viridis = cm.get_cmap('viridis', 12)
# print(viridis)
# print(viridis(0.56))

viridis = cm.get_cmap('rainbow', 256)
newcolors = viridis(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 0])
newcolors[:1, :] = pink
newcmp = ListedColormap(newcolors)

In [77]:
print(latitude)
print(xpixelsize)
print(cols)
print(longitude)
print(ypixelsize)
print(rows)

40.701497713586164
0.0002694945852358568
14
-8.420088821109099
-0.0002694945852358568
8


In [81]:
from folium.plugins import FloatImage
import base64
m = folium.folium.Map(location = (latitude,longitude),tiles = "Stamen Terrain",zoom_start=13)

# if not all data is 
#rows = 200
#cols = 200


# bounds = [[lat_min, lon_min], [lat_max, lon_max]]

folium.raster_layers.ImageOverlay(
    image=breaks_plot_years_norm,
    bounds=[[latitude, longitude], [latitude + rows*xpixelsize, longitude + cols*xpixelsize]],
    colormap = newcmp
).add_to(m)
img = "output/picture.png" 


####
resolution, width, height = 75, 4,4
encoded = base64.b64encode(open("output/picture.png", 'rb').read()).decode()
from folium import IFrame

html = '<img src="data:image/png;base64,{}">'.format
iframe = IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
popup = folium.Popup(iframe, max_width=2650)

icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[latitude, longitude], popup=popup, icon=icon)
marker.add_to(m)
####

m.save(os.path.join('output/ImageOverlay_5.html'))
m